<a href="https://colab.research.google.com/github/khare19yash/Automated-Detection-of-Neuropsychiatric-Disorders/blob/master/asd_dfc_ttest_rfecv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Mount

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd gdrive/My Drive/IIITH_Internship/Neuro/ASD-DFC

/content/gdrive/My Drive/IIITH_Internship/Neuro/ASD-DFC


### Install libraries

In [ ]:
!pip install nilearn

     |████████████████████████████████| 2.5MB 5.3MB/s 


In [ ]:
!pip install wandb

     |████████████████████████████████| 1.8MB 5.1MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 133kB 31.3MB/s 
     |████████████████████████████████| 163kB 30.3MB/s 
     |████████████████████████████████| 102kB 11.3MB/s 
     |████████████████████████████████| 71kB 10.8MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.3-cp36-none-any.whl size=73873 sha256=41abd85225c7377c2c5cbfaffa814caefe9c6c7e6d91674386e2a9c9144d2f64
  Stored in directory: /root/.cache/pip/wheels/a8/1d/38/2c19bb311f67cc7b4d07a2ec5ea36ab1a0a0ea50db994a5bc7
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=a5a850f9db1834513b47b6ec23a9b58345f5bc39edeed8af05a558c7faa654a8
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp36-none-any.whl size=8785 sha256=c5dfc73d7fd10c31b7b3ed569c199c26161fffc527a

In [ ]:
#imports

import os
import numpy as np
import pandas as pd
import math
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
# from nilearn import datasets
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import RFECV
# from nilearn import connectome
import torch.nn.functional as F
# from transformers import BertTokenizer, BertModel
from sklearn.decomposition import PCA
from scipy.stats import ttest_rel, ttest_ind
import shutil
import random
import numpy.ma as ma
from docopt import docopt
from functools import partial
from sklearn import preprocessing
from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
import wandb
from mpl_toolkits.mplot3d import Axes3D
from sklearn import decomposition
from sklearn import datasets
import warnings
warnings.simplefilter("ignore", UserWarning)

!wandb login 93ff2b234de4902b71d7405831d980f0c3182650

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


### Wandb

In [ ]:
DEBUG = False
RESUME = False
SEED = 101

In [ ]:

ID = wandb.util.generate_id() 
if RESUME:
    WANDB_ID = '162gtnyn'

wandb.init(project = 'asd3', name='ml-dfc', id='test')

config = wandb.config
config.epochs = 2 if DEBUG else 50           
config.bs =  4       # batch size
config.lr = 1e-4       # learning rateasd instalment
config.pat = 10         # patience for reduce lr on plateau
config.factor = 0.5    # factor by which to reduce lr
config.random_state = 101
config.window_size = 50
config.stride = 2
# config.in_channels = 19900
# config.out_channels = 32
# config.hidden_size = 768
config.hidden_dropout_prob = 0.3
config.n_layers = 1
config.heads = 2

wandb: Currently logged in as: pindi-krishna (use `wandb login --relogin` to force relogin)


### Data

In [ ]:
folder_path = 'CC200'
data_dir = '../ASD/Datasets/ASD_Dataset/'
phenotype_file_path = '../ASD/Datasets/ASD_Dataset/Phenotypic_V1_0b_preprocessed1.csv'

### Helper Functions

In [ ]:
#returns dataframe
def get_data(data_dir, folder_path, phenotype_file_path):
  file_paths = os.listdir(os.path.join(data_dir,folder_path))
  subject_ids = []
  for path in file_paths:
    index = path.find("5")
    subject_ids.append(int(path[index:index+5]))

  phenotype_df = pd.read_csv(phenotype_file_path)
  df = phenotype_df[phenotype_df['SUB_ID'].isin(subject_ids)]
  df = df[['SUB_ID','SITE_ID', 'DX_GROUP']]
  df.DX_GROUP = df.DX_GROUP.map({1: 1, 2:0})
  df['FILE_PATH'] = file_paths

  return df.values


def apply_threshold(data,data_dir,folder_path,threshold):
  n = data.shape[0]
  idx = []
  for i in range(n):
    path = os.path.join(data_dir,folder_path,data[i,3])
    timeseries_data = pd.read_csv(path, sep='\t')
    if timeseries_data.shape[0] > threshold:
        idx.append(i)
  return data[idx]

def confusion(g_turth,predictions):
    tn, fp, fn, tp = confusion_matrix(g_turth,predictions).ravel()
    accuracy = (tp+tn)/(tp+fp+tn+fn)
    sensitivity = (tp)/(tp+fn)
    specificty = (tn)/(tn+fp)
    return accuracy,sensitivity,specificty

#Utils
def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


def compute_connectivity(functional):
    with np.errstate(invalid="ignore"):
        corr = np.nan_to_num(np.corrcoef(functional))
        mask = np.invert(np.tri(corr.shape[0], k=-1, dtype=bool))
        m = ma.masked_where(mask == 1, mask)
        return ma.masked_where(m, corr).compressed()


def load_patient(data_dir, filepath, window=config.window_size,stride=config.stride, mode = 'sliding'):
    df = pd.read_csv(os.path.join(data_dir, filepath), sep="\t", header=0)
    df = df[:116]
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
    ROIs = ["#" + str(y) for y in sorted([int(x[1:]) for x in df.keys().tolist()])]
    functional = np.nan_to_num(df[ROIs].to_numpy().T)
    functional = preprocessing.scale(functional, axis=1)
    n_rois, n_samples = functional.shape
    n_slides = n_samples // window + 1
    dfc_matrix = []
    size = (n_samples - window) + 1
    for i in range(0, size, stride):

        df = functional[:, i : i + window]
        if(mode == 'tapered') :
            weights = np.random.randn(1, window)        # Normally distributed weights( Tapered Window)      
            df = weights * df
        elif(mode == 'weighted') :
            weights = np.linspace(1 / n_samples, window / n_samples, num = window) # Linearly distributed weights(Weighted Window)
            weights = weights[np.newaxis, : ]       
            df = weights * df
        fc = compute_connectivity(df)
        fc = fc.astype(np.float32)
#         fc = fc[:512]
        dfc_matrix.append(fc)
    return np.stack(dfc_matrix,axis=0)

def load_sfc(data_dir, X) : 
    sfc = []
    for filepath in X:
        df = pd.read_csv(os.path.join(data_dir, filepath), sep="\t", header=0)
        # df = df[:116]
        df = df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
        ROIs = ["#" + str(y) for y in sorted([int(x[1:]) for x in df.keys().tolist()])]
        functional = np.nan_to_num(df[ROIs].to_numpy().T)
        functional = preprocessing.scale(functional, axis=1)
        n_rois, n_samples = functional.shape
        # n_slides = n_samples // window + 1
        fc = compute_connectivity(functional)
        fc = fc.astype(np.float32)
        sfc.append(fc)
    return np.stack(sfc, axis = 0)

def load_dfc(data_dir, X, window = config.window_size,stride=config.stride, mode = 'tapered'):
    
    dfc = []
    for filepath in X:
        dfc.append(load_patient(data_dir, filepath, window=config.window_size,stride=config.stride, mode = mode))
    
    return np.stack(dfc,axis=0)
        
def apply_pca(dfc_data,n_components=1024):
    n_sample,n_window,n_feat = dfc_data.shape
    dfc_data = dfc_data.reshape((n_sample*n_window,n_feat))
    pca = PCA(n_components=n_components)
    dfc_data = pca.fit_transform(dfc_data)
    dfc_data = dfc_data.reshape((n_sample,n_window,-1))
    return dfc_data

def get_regs(datas):

    """
    extracts the features as proposed in asd diagnet paper. 
    returns data with relevant features and also region indices
    """
    n_sample, n_window, n_feat = datas.shape
    datas = datas.reshape((n_sample*n_window,n_feat))
    avg=[]
    for ie in range(datas.shape[1]):
        avg.append(np.mean(datas[:,ie]))
    avg=np.array(avg)
    highs=avg.argsort()[-regnum:][::-1]
    lows=avg.argsort()[:regnum][::-1]
    regions=np.concatenate((highs,lows),axis=0)
    data = datas[regions]
    dfc_data = data.reshape((n_sample,n_window,-1))
    return dfc_data, regions


### Get Data

In [ ]:
data = get_data(data_dir, folder_path, phenotype_file_path)
print(data.shape)       #tuple('SUB_ID','SITE_ID', 'DX_GROUP', 'FILE_PATH')

(1035, 4)


### Apply threshold on Time points for DFC

In [ ]:
data = apply_threshold(data,data_dir,folder_path,100)
print(data.shape)       #tuple('SUB_ID','SITE_ID', 'DX_GROUP', 'FILE_PATH')

(1009, 4)


### To get Paths and labels

In [ ]:
X = data[:, 3]      # Paths to inputs
Y = data[:, 2]      # labels
Y = Y.astype(np.float32)
print(X.shape)
print(Y.shape)
# del data

(1009,)
(1009,)


### Train-test-split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 30, stratify = Y)
print(X_train.shape)
print(X_test.shape)

(807,)
(202,)


In [ ]:
print(np.unique(Y_train, return_counts = True))
print(np.unique(Y_test, return_counts = True))

(array([0., 1.], dtype=float32), array([412, 395]))
(array([0., 1.], dtype=float32), array([103,  99]))


### 2 sample t test  SFC

In [ ]:
td_x, asd_x = [], []
td_y, asd_y = [], []
for i in range(len(X_train)):
    if(Y[i] == 0): 
        td_x.append(X_train[i])
        td_y.append(Y_train[i])
    else : 
      asd_x.append(X_train[i])
      asd_y.append(Y_train[i])

# print(len(asd_x), len(td_x))
data_dir = '../ASD/Datasets/ASD_Dataset/CC200'
td_fc = load_sfc(data_dir, td_x)
asd_fc = load_sfc(data_dir,asd_x)
fc = load_sfc(data_dir, X_train)

In [ ]:
t_values, p_values = [], []  
rel_ids = []   # relevant features indices 
for i in range(td_fc.shape[1]) :
    t, p = ttest_ind(td_fc[:, i : i+1], asd_fc[:, i : i+1], axis = 0, equal_var = False)
    t_values.append(t)
    p_values.append(p)
    if(p < 0.01) :
        rel_ids.append(i)

In [ ]:
len(rel_ids)

In [ ]:
rel_ids = np.array(rel_ids, dtype = 'uint8')
X_train = []
for i in range(len(fc)):
    p = fc[i]
    X_train.append(p[rel_ids])
X_train = np.array(X_train, dtype = 'float32')    
print(X_train.shape)

In [ ]:
print(np.unique(X, return_counts = True))
print(np.unique(Y, return_counts = True))

### 2 sample t test DFC 

In [ ]:
td_x, asd_x = [], []
td_y, asd_y = [], []
for i in range(len(X_train)):
    if(Y[i] == 0): 
        td_x.append(X_train[i])
        td_y.append(Y_train[i])
    else : 
      asd_x.append(X_train[i])
      asd_y.append(Y_train[i])

# print(len(asd_x), len(td_x))
data_dir = '../ASD/Datasets/ASD_Dataset/CC200'

td_fc = load_dfc(data_dir, td_x, mode = 'tapered')

asd_fc = load_dfc(data_dir,asd_x, mode = 'tapered')

fc = load_dfc(data_dir, X_train, mode = 'tapered')

In [ ]:
asd_fc = asd_fc.reshape(asd_fc.shape[0], np.prod(asd_fc.shape[1:]))
td_fc = td_fc.reshape(td_fc.shape[0], np.prod(td_fc.shape[1:]))
fc = fc.reshape(fc.shape[0], np.prod(fc.shape[1:]))

In [ ]:
print(asd_fc.shape)
print(td_fc.shape)

(397, 676600)
(410, 676600)


In [ ]:
t_values, p_values = [], []  
rel_ids = []   # relevant features indices 
for i in range(td_fc.shape[1]) :
    t, p = ttest_ind(td_fc[:, i : i+1], asd_fc[:, i : i+1], axis = 0, equal_var = False)
    t_values.append(t)
    p_values.append(p)
    if(p < 0.001) :
        rel_ids.append(i)

In [ ]:
rel_ids = np.array(rel_ids, dtype = 'uint8')
X_train = []
for i in range(len(fc)):
    p = fc[i]
    X_train.append(p[rel_ids])
X_train = np.array(X_train, dtype = 'float32')    
print(X_train.shape)

(807, 733)


In [ ]:
print(np.unique(X, return_counts = True))
print(np.unique(Y, return_counts = True))

(array(['CMU_a_0050642_rois_cc200.1D', 'CMU_a_0050646_rois_cc200.1D',
       'CMU_a_0050647_rois_cc200.1D', ..., 'Yale_0050626_rois_cc200.1D',
       'Yale_0050627_rois_cc200.1D', 'Yale_0050628_rois_cc200.1D'],
      dtype=object), array([1, 1, 1, ..., 1, 1, 1]))
(array([0., 1.], dtype=float32), array([515, 494]))


### Test

In [ ]:
test_fc = load_dfc(data_dir,X_test, mode = 'tapered')

In [ ]:
test_fc = test_fc.reshape(test_fc.shape[0], np.prod(test_fc.shape[1:]))

In [ ]:
rel_ids = np.array(rel_ids, dtype = 'uint8')
X_test = []
for i in range(len(test_fc)):
    p = test_fc[i]
    X_test.append(p[rel_ids])
X_test = np.array(X_test, dtype = 'float32')    
print(X_test.shape)

(202, 733)


### SFC

In [ ]:
data_dir = '../ASD/Datasets/ASD_Dataset/CC200'
fc = load_sfc(data_dir,X)
print(fc.shape)

(1035, 19900)


### DFC

In [ ]:
data_dir = '../ASD/Datasets/ASD_Dataset/CC200'
fc = load_dfc(data_dir,X)
print(fc.shape)

(1009, 34, 19900)


### Feature extraction using PCA

In [ ]:
fc = apply_pca(fc,n_components=1024)
print(fc.shape)

(1009, 34, 1024)


### Dimensionality Reduction proposed in ASD Diagnet Paper

In [ ]:
fc = get_regs(fc)
print(fc.shape)

### Feature selection using RFECV SFC


In [ ]:
from sklearn.feature_selection import RFECV
# X = fc.copy()
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(10), scoring='accuracy')
rfecv.fit(X_train, Y_train)
print("Optimal number of features : ", rfecv.n_features_)
print('Mask of features : ', rfecv.ranking_)

Optimal number of features :  102
Mask of features :  [ 1  1  1 23  1  1  1  1 14  1  1  7  1 17  1  1  1  1  1  1  1  1  1  1
  1  1  1 29 18 20  8  1  1  1  1  1  1  2  1  1 15  1  1  1  1  1  1 25
  1  1 26  1  6  1  1  1  1  1  1  1  1  1 28  1  1  1  1  1  1  1  1  1
  1 27 21  1  1 10  1  1  1  1  1  1  1  1 16  1  1  1  1  1  1  1  1  1
  4 22  1  1 11 13  1  1  1  3  1  1  1  1  1  1  9  1 24  1 12  1  1  1
  1  1  1  1 19  1  1  1  1  5]


In [ ]:
train_score = rfecv.score(X_train, Y_train)
test_score =  rfecv.score(X_test, Y_test)
print(train_score)
print(test_score)

0.6594202898550725
0.5652173913043478


### Feature selection using RFECV DFC

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
# X = fc.copy()
C = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
train = []
test = []
for c in C :
    svc = SVC(kernel="linear", C = c)
    rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(10), scoring='accuracy')
    rfecv.fit(X_train, Y_train)
    print("Optimal number of features : ", rfecv.n_features_)
    print('Mask of features : ', rfecv.ranking_)
    train_score = rfecv.score(X_train, Y_train)
    test_score =  rfecv.score(X_test, Y_test)
    train.append(train_score)
    test.append(test_score)
    print(train_score)
    print(test_score)

Optimal number of features :  511
Mask of features :  [ 59   1   1   1   1   1   1   1   1   1   1  64   6 103   1   1   1   1
  99   1   1   1   1 175   1   1   1   1   1   1   1   1  84   1   1   1
   1   1  52   1 223   1   1   1   1 107  77   1   1   1   1 200   1   1
   1   1  37   1   1   1   1  51  56   1 181 118   1   1   1   1   1   1
   1   1   1   1   1   1   1 204   1   1   1   1   1 137  75   1   1   1
 169   1   1 219   1   1  32  86   1 154  20 166   1  85   1 144   1 114
 104   1   1   1   1   1   1   1   1   1   1   1 213 206   1   1   1   1
   1 182   1   1   1   1   1 139 117   1   1   1   1 120   1 165  38  87
 214   1   1   1   1 209  10  39 143   1 101   1 124   1   7 185   1   1
   1   5   1   1   1   1   1   1   1   1 216  72   1   1   1   1 151   1
   1 146 189   1   1  80 202   1 195   1   1   1   1   1   1  66   1 194
   1   1 110 105   1 178 208   1   1  63   1   1   1   1   1   1   1   1
   1   1 157   1   1   1   1   1   1 179   1   1 205  44   1   1   1  

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV
# X = fc.copy()
C = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
train = []
test = []
for c in C :
    svc = SVC(kernel="linear", C = c)
    # rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(10), scoring='accuracy')
    svc.fit(X_train, Y_train)
    # print("Optimal number of features : ", rfecv.n_features_)
    # print('Mask of features : ', rfecv.ranking_)
    train_score = svc.score(X_train, Y_train)
    test_score =  svc.score(X_test, Y_test)
    train.append(train_score)
    test.append(test_score)
    print(train_score)
    print(test_score)

0.5142503097893433
0.5148514851485149
0.6815365551425031
0.5742574257425742
0.7496902106567535
0.504950495049505
0.7682775712515489
0.5
0.7856257744733581
0.4900990099009901
0.7868649318463445
0.49504950495049505


In [ ]:
X_train.shape

(34, 174799)

In [ ]:
print('Best params')
print(clf.best_params_)
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

### Flattening input

In [ ]:
X = fc.reshape(fc.shape[0], np.prod(fc.shape[1:]))
print(X.shape)

(1009, 34816)


### ML Approach

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


In [ ]:
svc = SVC()
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=32)
parameters = {'kernel':('linear', 'rbf', 'linear', 'poly', 'rbf', 'sigmoid'), 'C':[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}

clf = GridSearchCV(estimator = svc, param_grid = parameters, cv = skfold, return_train_score=True)
clf.fit(X, Y)

In [ ]:
print(clf.best_params_)

In [ ]:
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
          % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()

### Simple train-test split and training linear svm

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 32, stratify = Y)
C = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
for c in C : 
    clf = SVC(C = c)
    clf.fit(X_train, Y_train)
    train_acc = clf.score(X_train, Y_train)
    test_acc = clf.score(X_test, Y_test)
    print(train_acc, test_acc)


0.5105328376703842 0.5099009900990099
0.5105328376703842 0.5099009900990099
0.5105328376703842 0.5099009900990099
0.9975216852540273 0.49504950495049505
1.0 0.48514851485148514
1.0 0.48514851485148514


### Data Loader

In [ ]:
class ASDDataset():
  def __init__(self,x,y,data_dir):
    self.x = x
    self.y = y
    self.data_dir = data_dir

  def __len__(self):
    return len(self.x)

  def __getitem__(self, item_idx):
#     filepath = self.x[item_idx]
#     x = torch.tensor(load_patient(self.data_dir,filepath),dtype=torch.float)
#     y = torch.tensor(self.y[item_idx],dtype=torch.float)

    return torch.tensor(self.x[item_idx],dtype=torch.float), torch.tensor(self.y[item_idx],dtype=torch.float)

### Train Test Functions

In [ ]:

def loss_fn(logits, targets):
  return nn.BCEWithLogitsLoss()(logits,targets)

def train_loop_fn(data_loader, model, optimizer, device, scheduler=None):
  model.train()
  train_loss = []
  final_targets = []
  final_predictions = []
  bar = tqdm(data_loader,leave=False)
  for (inputs, targets) in bar:
    inputs, targets = inputs.to(device), targets.to(device)
    optimizer.zero_grad()
    logits = model(inputs)
#     print(logits.shape)
#     print(targets.shape)
    loss = loss_fn(logits , targets)
    train_loss.append(loss.detach().cpu().numpy())
    
    proba = torch.sigmoid(logits).detach().cpu().numpy()
    predictions = np.ones_like(proba, dtype=np.int32)
    predictions[proba < 0.5] = 0
    final_targets.append(targets.detach().cpu().numpy())
    final_predictions.append(predictions)
    
    loss.backward()
    optimizer.step()

    if scheduler is not None:
      scheduler.step()

  final_targets = np.concatenate(final_targets)
  final_predictions = np.concatenate(final_predictions)
  train_accuracy = np.mean(final_targets == final_predictions)
  mlp_acc,mlp_sens,mlp_spef = confusion(final_targets,final_predictions)

    
  
  return np.mean(train_loss), mlp_acc

def eval_fn(data_loader, model, device):
    model.eval()
    eval_loss = []
    final_targets = []
    final_predictions = []
    bar = tqdm(data_loader,leave=False)
    
    with torch.no_grad():
      for (inputs, targets) in bar:
        inputs, targets = inputs.to(device), targets.to(device)
        logits = model(inputs)
        print(logits.shape)
        print(targets.shape)
        loss = loss_fn(logits , targets)
        eval_loss.append(loss.detach().cpu().numpy())
        
        proba = torch.sigmoid(logits).detach().cpu().numpy()
        predictions = np.ones_like(proba, dtype=np.int32)
        predictions[proba < 0.5] = 0
        
        final_predictions.append(predictions)
        final_targets.append(targets.detach().cpu().numpy())
        
    final_targets = np.concatenate(final_targets)
    final_predictions = np.concatenate(final_predictions)
    mlp_acc,mlp_sens,mlp_spef = confusion(final_targets,final_predictions)
    metrics_dict = {'accuracy': np.round(mlp_acc, 4), 
                        'senstivity' : np.round(mlp_sens,4), 
                        'specificity' : np.round(mlp_spef,4), 
                        'loss' : np.round(np.mean(eval_loss),4)}

    accuracy = np.mean(final_targets == final_predictions)
    eval_loss = np.mean(eval_loss)

    return accuracy, metrics_dict

def run_k_fold(X,Y,num_splits=10):
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  data_dir = '../ASD/Datasets/ASD_Dataset/CC200'

  if num_splits is not None:
    # list to store metrics after each fold
    crossval_acc=[]
    crossval_sen=[]
    crossval_spec=[]
    crossval_loss=[]
    skf = StratifiedKFold(n_splits=num_splits, random_state=1, shuffle=True) 
    kk = 0
    for train_index, test_index in skf.split(X, Y):
        #init wandb 
        NAME = f'asd-cnnlstm-fold-{kk+1}'
        ID = f'cnnlstm-fold-{kk+1}'
#         wandb.init(project = 'asd1', name=NAME, id=ID)

        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]


        verbose = (True if (kk == 0) else False)

        train_dataset = ASDDataset(x_train,y_train, data_dir)
        test_dataset = ASDDataset(x_test,y_test, data_dir)

        train_dataloader = DataLoader(train_dataset,batch_size=config.bs,shuffle=True)
        test_dataloader = DataLoader(test_dataset,batch_size=config.bs,shuffle=False)

        """
        :param emb: Embedding dimension
        :param heads: nr. of attention heads
        :param depth: Number of transformer blocks
        :param seq_length: Expected maximum sequence length
        :param num_tokens: Number of tokens (usually words) in the vocabulary
        :param num_classes: Number of classes.
        :param max_pool: If true, use global max pooling in the last layer. If false, use global
                         average pooling.
        """

        model = CTransformer(1024, 8, 20, 4, 4, 1,device)  # (embedding dimension, #heads, depth, seq_length(4), num_tokens(4), num_classes)
        model.to(device)
#         wandb.watch(model, log='all')      # this logs everything in wandb account
        optimizer = optim.Adam(model.parameters(),lr=config.lr)

        for epoch in range(config.epochs):

            
            train_loss, train_accuracy = train_loop_fn(train_dataloader,model,optimizer,device)                                                   
            print(f'Epoch {epoch + 1}/{config.epochs}')

            wandb.log({'Train loss': train_loss,
                        'Train accuracy': train_accuracy})

            content = f'Classifier Train loss: {(train_loss):.4f}, Train Acc: {(train_accuracy):.4f}'
            print(content)

        accuracy, metrics_dict = eval_fn(test_dataloader, model, device)
        print("-----------------------------")
        print(f'Fold {kk+1}/{num_splits}')
        content = f'{metrics_dict}'
        print(content)
        print("-----------------------------")

        crossval_acc.append(metrics_dict['accuracy'])
        crossval_sen.append(metrics_dict['senstivity'])
        crossval_spec.append(metrics_dict['specificity'])
        crossval_loss.append(metrics_dict['loss'])

        #save the model after each fold
        recorder = {'optimizer': optimizer.state_dict(),
        'model': model.state_dict(),
        'fold' : kk+1}

        torch.save(recorder, f'./Models/{NAME}.pt')
        
        kk = kk + 1

    print("*********************************")    
    print(f'Average Value after 10 Folds------->')
    content = f'Accuracy: {np.round(np.mean(crossval_acc),4)}, Senstivity: {np.round(np.mean(crossval_sen),4)}, Specificity: {np.round(np.mean(crossval_spec),4)}, Loss: {np.round(np.mean(crossval_loss),4)}'
    print(content)
    print("*********************************") 

In [ ]:
run_k_fold(dfc,Y)

Epoch 1/50
Classifier Train loss: 0.8620, Train Acc: 0.4769


Epoch 2/50
Classifier Train loss: 0.7189, Train Acc: 0.5000


Epoch 3/50
Classifier Train loss: 0.7148, Train Acc: 0.5022


Epoch 4/50
Classifier Train loss: 0.7153, Train Acc: 0.4890


Epoch 5/50
Classifier Train loss: 0.7067, Train Acc: 0.5132


...

Epoch 27/50
Classifier Train loss: 0.6968, Train Acc: 0.4868


Epoch 28/50
Classifier Train loss: 0.6957, Train Acc: 0.4934


Epoch 29/50
Classifier Train loss: 0.6973, Train Acc: 0.5132


Epoch 30/50
Classifier Train loss: 0.6953, Train Acc: 0.4978


KeyboardInterrupt: ignored